<h1><b>Restricted Boltzmann Machine</b></h1>
<p align="justify">Στην συγκεκριμένη άσκηση θα μελετήσετε τον τρόπο λειτουργίας μιας <i>RBM (<a href="https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine">Restricted Boltzmann Machine</a>)</i>, χρησιμοποιώντας το έτοιμο πρόγραμμα που σας δίνεται.Το συγκεκριμένο πρόγραμμα αξιοποιεί το <a href="https://en.wikipedia.org/wiki/MNIST_database">dataset του <i>MNIST</i></a>, όπου είναι μια μεγάλη βάση δεδομένων με χειρόγραφα ψηφία που χρησιμοποιείται συνήθως για την εκπαίδευση διαφόρων συστημάτων επεξεργασίας εικόνας.</p>
<p align="justify">Μία αρκετά σημαντική εφαρμογή της <i>RBM</i> είναι η εξαγωγή χαρακτηριστικών (feature representation) από ένα dataset με σκοπό την αναπαράσταση της εισόδου (ορατοί νευρώνες) με ένα διάνυσμα μικρότερης διάστασης (κρυφοί νευρώνες). Στη συγκεκριμένη άσκηση θα συγκρίνετε την ακρίβεια ενός ταξινομητή ψηφίων με τη χρήση του αλγορίθμου <i>Logistic Regression</i>, όταν εκείνος δέχεται ως είσοδο το dataset (i) αφου υποστεί επεξεργασία από το <i>RBM</i> και (ii) χωρίς να έχει υποστεί επεξεργασία από το <i>RBM</i>.</p>
<p align="justify">Το δοθέν <i>dataset</i>, στα πλαίσια της άσκησης έχει
διογκωθεί με τεχνητό τρόπο (γραμμική μετατόπιση ενός εικονοστοιχείου (pixel) σε
κάθε κατεύθυνση) ώστε να έχουμε ένα <i>dataset</i> 5 φορές μεγαλύτερο. Με βάση τον κώδικα που σας έχει δοθεί, καλείστε να απαντήσετε στα παρακάτω ερωτήματα:</p>
<ul>
<li>1. Να περιγράψετε σύντομα τον τρόπο λειτουργίας μιας <i>RBM</i>. Τι διαφορές έχει σε σχέση με μία <i> Μηχανή Boltzmann</i>;</li>
<li>2. Να αναφέρετε τις βασικότερες εφαρμογές μιας RBM.</li>
<li>3. Συγκρίνετε τα αποτελέσματα της ταξινόμησης με τον αλγόριθμo <i>Logistic Regression</i> χωρίς τη χρήση RBM σε σχέση με τα αποτελέσματα της ταξινόμησης που έχει χρησιμοποιηθεί η <i>RBM</i> για την εξαγωγή των χαρακτηριστικών. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;</li>
<li>4. Επιχειρήστε να αλλάξετε τον αριθμό των κρυφών νευρώνων (components) από 100 σε 200. Τι
παρατηρείτε ως προς τα αποτελέσματα και τον χρόνο που έτρεξε το
πρόγραμμα μέχρι να γίνει <i>fit</i>΄;</li>
<li>5. Προσπαθήστε να τρέξετε το πρόγραμμα τρέξτε το πρόγραμμα εκ νέου με το <i>κανονικό dataset</i>. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων; Πού μπορεί να οφείλονται τυχόν αποκλίσεις σε σχέση με πριν;</li>
</ul>

In [ ]:
from __future__ import print_function

print(__doc__)

# Authors: Yann N. Dauphin, Vlad Niculae, Gabriel Synnaeve
# License: BSD

import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone
import time 


# #############################################################################
# Setting up

def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    def shift(x, w):
        return convolve(x.reshape((8, 8)), mode='constant', weights=w).ravel()

    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y


# Load Data
digits = datasets.load_digits()
X = np.asarray(digits.data, 'float32')
X, Y = nudge_dataset(X, digits.target)
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

# Models we will use
logistic = linear_model.LogisticRegression(solver='lbfgs', max_iter=10000,
                                           multi_class='multinomial')
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

# #############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.06
rbm.n_iter = 20
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000


# Training RBM-Logistic Pipeline
start = time.time()
rbm_features_classifier.fit(X_train, Y_train)
total_time = time.time() - start 

print('********************************************')
print(' RBM Training Time with LogReg: ', total_time)
print('********************************************')

# Training the Logistic regression classifier directly on the pixel
raw_pixel_classifier = clone(logistic)
raw_pixel_classifier.C = 100.
raw_pixel_classifier.fit(X_train, Y_train)

# #############################################################################
# Evaluation

Y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

Y_pred = raw_pixel_classifier.predict(X_test)
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

# #############################################################################
# Plotting

plt.figure(figsize=(4.2, 4))
for i, comp in enumerate(rbm.components_):
    plt.subplot(10, 10, i + 1)
    plt.imshow(comp.reshape((8, 8)), cmap=plt.cm.gray_r,
               interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
plt.suptitle('100 components extracted by RBM', fontsize=16)
plt.subplots_adjust(0.08, 0.02, 0.92, 0.85, 0.08, 0.23)

plt.show()

## Aπαντήσεις:

1. **RBM**: Στοχαστικοί νευρώνες 2 επιπέδων (ορατό & κρυφό), με συμμετρικές συνάψεις και καταστάσεις [0,1]. Οι καταστάσεις των ορατών νευρώνων κωδικοποιούν χαρακτηριστικά $v_i$ τα οποία είναι παρατηρήσιμα (δειγματα εισόδου-εξόδου) ενώ του κρυφού επιπέδου $h_j$ κωδικοποιούν κρυφές ιδιότητες. Η συνάρτηση ενεργοποίησης είναι η Sigmoid και τα δεδομένα την διαπερνούν μετά απο άθροιση των συναπτικών βαρών και τον bias και των 2 καταστασεων νευρώνων (ορατών & κρυφών).  Η κύρια διαφορά με το **ΒΜ** έχει να κάνει με την συνδεσιμότητα των νευρώνων των ιδίων επιπέδων όπου στο **RΒΜ**  είναι ασύνδετοι επιταχύνοντας έτσι την παραγωγή δείγματος με στατιστική ομοιότητα με τα δεδομένα μάθησης. 

2. Βασικότερες Εφαρμογές RBM:
* dimensionality reduction
* classification
* collaborative filtering 
* feature learning 
* Incorporate them in Deep Learning Architecture

3. Τα αποτελέσματα με χρήση της Logistic Regression & RBM είναι σαφώς καλύτερα από ότι με την απλή Logistic Regression. H χρήση RBM στην περίπτωση αυτή  λειτουργεί ως feature extraxtion τα οποία οδηγούν σε σωστότερες αποφάσεις-προβλέψεις. Επίσης να σημειωθεί ότι η χρήση της RBM αντιμετωπίζει καλύτερα το συγκεκριμένο datasetοπου έχει εφαρμοστέι data augmentation με την χρήση shifts. Στην περίπτωση της απλής Logistic Regression αυτός ο εμπλουτισμός σε πολλές περιπτώσεις λειτουργεί ως θόρυβος στα δεδομένα καθώς είναι πιθανόν να τροποποιεί και την εικόνα τους και συνεπώς να οδηγεί σε λανθασμένες προβλέψεις.

4. Η αύξηση των νευρώνων (100->200) δεν συντελεί σε βελτίωση των προβλεψεων παρα μόνον σε αύξηση του χρόνου εκπαίδευσης. Ένα συμπερασμα θα μπορούσε να είναι ότι οι υπάρχοντες (νευρωνες) αρκούν για την εκτίμηση του συγκεκριμένου ντατασετ και μιας και ασύνδετοι, η αύξηση του αριθμού τους προσφέρει μονάχα υπολογιστική πολυπλοκότητα.

5. Όπως επισημάναμε και στην απάντηση 3 παρατηρούμε ότι το Data Augmentation δεν οδήγησε σε σημαντικη βελτίωση του μοντέλου με RBM και αντιθέτως στη περίπτωση με το απλό Logistic Regression οδήγησε σε αρκετά χειρότερα αποτελέσματα. Χωρίς data augmentation το Logistic Regression πέτυχε πολυ καλύτερα αποτελέσματα.